In [6]:

%xmode Minimal

#  Setting display options 

import pandas as pd
import numpy  as np
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate


print("--")


Exception reporting mode: Minimal


--


In [7]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#########################################################################


In [ ]:


%load_ext autoreload
%autoreload 2


from katana import remote
#  from katana.remote import export_data, import_data


my_client = remote.Client()
print(my_client.server_version)

print()
print("--")



In [ ]:

from config import hyperparams


input_config = hyperparams.load_input_config()
print(input_config)


print()
print("--")



In [8]:

#     from timeit import default_timer
#     
#     # Import the module that uses Dask to import drug data
#     from src import dask_ingestion
#     
#     graph = client.create_graph(num_partitions=input_config.num_partitions)


#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#     if input_config.use_train_rdg:
#         print(f"Import pretrained graph from: {input_config.trained_rdg_path}")
#         import_data.rdg(graph, input_config.trained_rdg_path)
#     else:
#         print("Generate the graph with data from source")
#         dask_ingestion.generate_deepcdr_graph(graph, input_config)



<_Graph my_graph, EVW1P2kCKvso8wrkChZF1jr25YumM78N3X97EFW693n9, 5>


In [9]:


from src import katana_pipeline

#  This method returns a handle only, no processing ..
#
rec_pipeline = katana_pipeline.RecipePipeline(my_graph)


#  rec_pipeline.graph.schema().view()



This ran MMM


In [ ]:

#########################################################################


In [ ]:


#  start_time = default_timer()

rec_pipeline.feature_generator()

#  print(f"***Took {default_timer() - start_time} seconds to generate the features.***")


#  feature_generator() runs,
#
#     
#       features = {
#           "HAS_EXPRESSION_OBSERVATION": "genomics_expression",
#           "HAS_METHYLATION_OBSERVATION": "genomics_methylation",
#           "HAS_MUTATION_OBSERVATION": "genomics_mutation",
#       }
#       for u, v in features.items():
#           preprocessing.set_cell_line_property(self.graph, u, v)

#       preprocessing.remove_null_cells(self.graph)
#       preprocessing.delete_single_node(self.graph)






In [ ]:
stats = rec_pipeline.stats()
stats

In [ ]:
start_time = default_timer()
rec_pipeline.split_generator(input_config)
print(f"***Took {default_timer() - start_time} seconds to generate the split.***")
rec_pipeline.graph.schema().view()

In [ ]:
model_config = hyperparams.load_model_config()
model_config

In [ ]:
training_config = hyperparams.load_training_config()
training_config

In [ ]:
start_time = default_timer()
validation_metric = rec_pipeline.train(model_config, training_config)
print(f"***Took {default_timer() - start_time} seconds to train the model.***")
print("Validation metric: ", validation_metric)

In [ ]:
start_time = default_timer()
metrics, predictions, labels = rec_pipeline.test(training_config)
print(f"***Took {default_timer() - start_time} seconds to test the model.***")
metrics

In [ ]:
start_time = default_timer()
rec_pipeline.plot(labels, predictions)
print(f"***Took {default_timer() - start_time} seconds to plot figures.***")

In [ ]:
rec_pipeline.infer(training_config)

In [ ]:
bortezomib = "B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN=C2)(O)O"
cell_line = "ACH-000001"
rec_pipeline.infer(training_config, drug=bortezomib, cell_line=cell_line)

In [ ]:
start_time = default_timer()
rec_pipeline.infer_embeddings(model_config)
print(f"***Took {default_timer() - start_time} seconds to save node embeddings.***")
rec_pipeline.graph.schema().view()

In [ ]:

#########################################################################


In [ ]:

#  if input_config.save_graph_path:
#      export_data.rdg(graph, input_config.save_graph_path)
    
    